In [1]:
import requests
import time
import datetime
import warnings
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys
warnings.filterwarnings("ignore")

In [2]:
def get_file(query):
    driver_path = r"./chromedriver.exe"
    driver = webdriver.Chrome(driver_path)

    url = "http://www.cboe.com/delayedquote/quote-table-download"
    driver.get(url)
    time.sleep(2 + random.random())

    soup = BeautifulSoup(driver.page_source, "html.parser")
    input_element = driver.find_element_by_xpath('//*[@id="txtTicker"]')
    input_element.send_keys(query)
    input_element.send_keys(Keys.ENTER)
    time.sleep(10 + random.random())
    driver.quit()

In [6]:
def read_this_month(path):
    file = open(path, 'r').read().split('\n')
    file = file[3: ]

    temp_list = []
    for element in file:
        temp_list.append(element.split(","))
    df = pd.DataFrame(temp_list)
    df = df.iloc[:, [0, 11, 10, 11, 21]]
    df.columns = ["Expiration Date", "Strike", "Call", "Strike", "Put"]
    
    this_month = int(pd.to_datetime('today').month)
    df_this_month = df.copy()

    df_this_month = df_this_month.dropna(axis=0)
    df_this_month["Date"] = df_this_month["Expiration Date"].apply(lambda x: pd.to_datetime(x))
    df_this_month["Date"] = df_this_month["Date"].apply(lambda x: str(x.date()))
    df_this_month['Date'] = df_this_month['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df_this_month['Month'] = df_this_month['Date'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
    df_this_month = df_this_month[df_this_month["Month"] == this_month]
    df_this_month.set_index(df_this_month['Date'], drop=True, inplace=True)
    # Drop the date before today
    df_this_month = df_this_month.loc[pd.to_datetime('today').date():]
    df_this_month.reset_index(drop=True, inplace=True)
    df_this_month["Call"] = df_this_month["Call"].apply(lambda x: int(x))
    df_this_month["Put"] = df_this_month["Put"].apply(lambda x: int(x))
    df_call_1 = df_this_month.groupby(df_this_month.iloc[:, 1])["Call"].agg(sum).sort_values(ascending=False)
    df_call_1 = pd.DataFrame(df_call_1)
    df_call_1.reset_index(drop=False, inplace=True)
    df_put_1 = df_this_month.groupby(df_this_month.iloc[:, 1])["Put"].agg(sum).sort_values(ascending=False)
    df_put_1 = pd.DataFrame(df_put_1)
    df_put_1.reset_index(drop=False, inplace=True)
    df_1 = pd.concat([df_call_1, df_put_1], axis=1)
    
    return df_1

def read_next_month(path):
    file = open(path, 'r').read().split('\n')
    file = file[3: ]

    temp_list = []
    for element in file:
        temp_list.append(element.split(","))
    df = pd.DataFrame(temp_list)
    df = df.iloc[:, [0, 11, 10, 11, 21]]
    df.columns = ["Expiration Date", "Strike", "Call", "Strike", "Put"]
    
    next_month = int(pd.to_datetime('today').month) + 1
    df_next_month = df.copy()
    
    df_next_month = df_next_month.dropna(axis=0)
    df_next_month["Date"] = df_next_month["Expiration Date"].apply(lambda x: pd.to_datetime(x))
    df_next_month["Date"] = df_next_month["Date"].apply(lambda x: str(x.date()))
    df_next_month['Date'] = df_next_month['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df_next_month['Month'] = df_next_month['Date'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
    df_next_month = df_next_month[df_next_month["Month"] == next_month]
    df_next_month.set_index(df_next_month['Date'], drop=True, inplace=True)
    # Drop the date before today
    df_next_month = df_next_month.loc[pd.to_datetime('today').date():]
    df_next_month.reset_index(drop=True, inplace=True)   
    df_next_month["Call"] = df_next_month["Call"].apply(lambda x: int(x))
    df_next_month["Put"] = df_next_month["Put"].apply(lambda x: int(x))
    df_call_2 = df_next_month.groupby(df_next_month.iloc[:, 1])["Call"].agg(sum).sort_values(ascending=False)
    df_call_2 = pd.DataFrame(df_call_2)
    df_call_2.reset_index(drop=False, inplace=True)
    df_put_2 = df_next_month.groupby(df_next_month.iloc[:, 1])["Put"].agg(sum).sort_values(ascending=False)
    df_put_2 = pd.DataFrame(df_put_2)
    df_put_2.reset_index(drop=False, inplace=True)
    df_2 = pd.concat([df_call_2, df_put_2], axis=1)
    
    return df_2

In [7]:
df_this_month = read_this_month(path=r"C:\Users\YangWang\Downloads\quotedata.dat")
df_this_month

,Strike,Call,Strike,Put
0,330.000,28125,290.000,59213
1,325.000,20331,310.000,50118
2,335.000,17070,240.000,50083
3,310.000,13354,300.000,40715
4,318.000,12745,255.000,37365
...,...,...,...,...
113,225.000,0,354.000,0
114,356.000,0,353.000,0
115,272.000,0,352.000,0
116,273.000,0,351.000,0


In [8]:
df_next_month = read_next_month(path=r"C:\Users\YangWang\Downloads\quotedata.dat")
df_next_month

,Strike,Call,Strike,Put
0,310.000,82720,310.000,117325
1,330.000,68534,280.000,109022
2,335.000,56402,265.000,107360
3,320.000,50063,290.000,103721
4,305.000,45411,260.000,103202
...,...,...,...,...
163,227.000,0,357.000,0
164,356.000,0,356.000,0
165,165.000,0,354.000,0
166,160.000,0,415.000,0


In [16]:
def certain_date_option(path, input_date):
    file = open(path, 'r').read().split('\n')
    file = file[3: ]

    temp_list = []
    for element in file:
        temp_list.append(element.split(","))
    df = pd.DataFrame(temp_list)
    df = df.iloc[:, [0, 11, 10, 11, 21]]
    df.columns = ["Expiration Date", "Strike", "Call", "Strike", "Put"]
    
    this_month = int(pd.to_datetime('today').month)
    df_temp = df.copy()

    df_temp = df_temp.dropna(axis=0)
    df_temp["Date"] = df_temp["Expiration Date"].apply(lambda x: pd.to_datetime(x))
    df_temp["Date"] = df_temp["Date"].apply(lambda x: str(x.date()))
    df_temp['Date'] = df_temp['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    df_temp['Month'] = df_temp['Date'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
    df_temp = df_temp[df_temp["Month"] == this_month]
    df_temp.set_index(df_temp['Date'], drop=True, inplace=True)
    # Drop the date before today
    df_temp = df_temp.loc[pd.to_datetime('today').date():]
    df_temp = df_temp[df_temp["Date"] == pd.to_datetime(input_date)]
    print(df_temp)
    df_temp.reset_index(drop=True, inplace=True)
    df_temp["Call"] = df_temp["Call"].apply(lambda x: int(x))
    df_temp["Put"] = df_temp["Put"].apply(lambda x: int(x))
    df_call = df_temp.groupby(df_temp.iloc[:, 1])["Call"].agg(sum).sort_values(ascending=False)
    df_call = pd.DataFrame(df_call)
    df_call.reset_index(drop=False, inplace=True)
    df_put = df_temp.groupby(df_temp.iloc[:, 1])["Put"].agg(sum).sort_values(ascending=False)
    df_put = pd.DataFrame(df_put)
    df_put.reset_index(drop=False, inplace=True)
    df_final = pd.concat([df_call, df_put], axis=1)
    
    return df_final

In [17]:
df_temp = certain_date_option(path=r"C:\Users\YangWang\Downloads\quotedata (2).dat", input_date="2020-07-31")
df_temp

           Expiration Date   Strike  Call   Strike   Put       Date  Month
Date                                                                      
2020-07-31      07/31/2020  155.000     0  155.000   162 2020-07-31      7
2020-07-31      07/31/2020  160.000     0  160.000  1183 2020-07-31      7
2020-07-31      07/31/2020  165.000     0  165.000    51 2020-07-31      7
2020-07-31      07/31/2020  170.000     0  170.000    83 2020-07-31      7
2020-07-31      07/31/2020  175.000     0  175.000   809 2020-07-31      7
...                    ...      ...   ...      ...   ...        ...    ...
2020-07-31      07/31/2020  375.000  1080  375.000     0 2020-07-31      7
2020-07-31      07/31/2020  380.000   467  380.000     0 2020-07-31      7
2020-07-31      07/31/2020  385.000   153  385.000     0 2020-07-31      7
2020-07-31      07/31/2020  390.000  1215  390.000     5 2020-07-31      7
2020-07-31      07/31/2020  395.000     0  395.000     0 2020-07-31      7

[119 rows x 7 columns]


,Strike,Call,Strike,Put
0,330.000,32304,310.000,75280
1,325.000,23557,315.000,50436
2,339.000,22761,290.000,50359
3,320.000,13907,240.000,49800
4,310.000,12981,270.000,48881
...,...,...,...,...
114,215.000,0,349.000,0
115,272.000,0,348.000,0
116,273.000,0,347.000,0
117,356.000,0,346.000,0


In [14]:
get_file(query="SPY")

In [21]:
path = "quote.dat "
path = path.lstrip().rstrip()